[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PennNGG/Quantitative-Neuroscience/blob/master/Answers%20to%20Exercises/Python/Frequentist%20Versus%20Bayesian%20Approaches%20Exercise%20Answers.ipynb)

Below are answers to the exercises described [here](https://github.com/PennNGG/Quantitative-Neuroscience/blob/master/Concepts/Python/Frequentist%20Versus%20Bayesian%20Approaches.ipynb)

In [2]:
import scipy.stats as st
import numpy as np

### Exercise #1: If someone gets a positive test, is it "statistically significant" at the p<0.05 level? Why or why not?

**Answer**: Statistical significance from the Frequentist perspective is typically measured by comparing $p$ to a threshold value; e.g., $p<0.05$. In this case, $p$ is shorthand for "the probabilty of obtaining the data under the Null Hypothesis", so we are checking for: $$p(data\,|\,Null\,Hypothesis) < 0.05$$
Here we take the Null Hypothesis as "not infected", and the data are just the single positive test. Therefore, the relvant p-value is simply the false-positive rate: $p=0.05$, which is typically considered "not significant." However, you can also see that it is not particularly informative.

In [3]:
N = 10000                   # size of the SAMPLE
false_positive_rate = 0.05 
false_negative_rate = 0

print(f'The probability of obtaining the data under the Null Hypothesis = {false_positive_rate}')

The probability of obtaining the data under the Null Hypothesis = 0.05


### Question #2: What is the probability that if someone gets a positive test, that person is infected?

**Answer**: Here we are asking for a different probability: $$p(infected\,|\,positive\,test) = p(hypothesis\,|\,data)$$ which is the "posterior probability" of the hypothesis, given the data.

Let's work our way backwards to figure out what information we need to solve this problem. We can compute the probability that someone with a positive test is infected from a particular population as:$$probability\,infected\,given\,positive\,test = \frac{total(infected\,and\,postive)}{total(positive)}$$

It should be obvious that to compute this quantity, we need to know the number of people in the population who are actually infected, in addition to knowing the number of people who had a positive test.

For this exercise, assume a range of priors (infection rates) from 0 to 1 in steps of 0.1.

In [4]:
# So let's start by defining how many in the population are actually infected. 
# We'll start by assuming that that *real* rate of infection is 0.5 (i.e., 
# half the POPULATION is infected), and then do a quick simulation to find 
# out how many in our SAMPLE of N people are infected. We can do this 
# simulation by getting N picks from a binomial distribution, where 
# each pick determines "isInfected" for a single person according to the 
# assumed rate of infection:
is_infected = st.binom.rvs(1, 0.5, size=N)

# Now we can count the number infected
num_infected = is_infected.sum()

# Now we need to count the number of people who got a positive test in this 
# example. There is no false negative rate, which implies that everyone who 
# is infected got a positive test:
is_positive = np.copy(is_infected)

# But there is a non-zero false-positive rate, which implies that some of the 
# people who are **not** infected will also have a positive test. We can use 
# binornd again to generate random picks from a binomial distribtuion according 
# to the false-positive rate:
is_positive[is_infected==0] = st.binom.rvs(1, false_positive_rate, size=N-num_infected)

# Now we can compute the probability that someone with a positive test is infected:
p_is_infected_given_is_positive = (np.logical_and(is_infected==1, is_positive==1).sum())/is_positive.sum()
print(f'Probaility infected given a positive test = {p_is_infected_given_is_positive:.4f}')

Probaility infected given a positive test = 0.9502


In [5]:
# Now we can do the same thing for the range 0->1
infected_proportions = np.arange(0.0, 1.1, 0.1)
for idx, val in enumerate(infected_proportions):
    
   # Simulate # infections in the SAMPLE, given the POPULATION rate
   is_infected = st.binom.rvs(1, val, size=N)
      
   # Count the number infected
   num_infected = is_infected.sum()
   
   # Make array of positive tests, given that falseNegativeRate=0 ...
   is_positive = np.copy(is_infected)
   
   # And falsePositiveRate > 0
   is_positive[is_infected==0] = st.binom.rvs(1, false_positive_rate, size=N-num_infected)
   
   # The probability that someone with a positive test is infected
   p_is_infected_given_is_positive = (np.logical_and(is_infected==1, is_positive==1).sum())/is_positive.sum()
   
   # We can compute the Bayesian Posterior as:
   # p(hypothesis | data) = (p(data | hypothesis) * p(hypothesis)) / p(data)
   # Note that we are using the true rate from the full POPULATION, so these predictions will differ slightly from the probability computed above (pIsInfectedGivenIsPositiveTest) from the SAMPLE
   p_data_given_hypothesis = 1 - false_negative_rate
   p_hypothesis = val
   p_data = is_positive.sum()/is_positive.size   
   p_hypothesis_given_data = (p_data_given_hypothesis * p_hypothesis) / p_data

   # Compute the theoretial posterior probability: 
   print(f'Infection rate={val:.1f}, proportion infected given a positive test={p_is_infected_given_is_positive:.3f}, Posterior={p_hypothesis_given_data:.3f}')

Infection rate=0.0, proportion infected given a positive test=0.000, Posterior=0.000
Infection rate=0.1, proportion infected given a positive test=0.716, Posterior=0.725
Infection rate=0.2, proportion infected given a positive test=0.832, Posterior=0.831
Infection rate=0.3, proportion infected given a positive test=0.903, Posterior=0.908
Infection rate=0.4, proportion infected given a positive test=0.930, Posterior=0.921
Infection rate=0.5, proportion infected given a positive test=0.955, Posterior=0.952
Infection rate=0.6, proportion infected given a positive test=0.965, Posterior=0.979
Infection rate=0.7, proportion infected given a positive test=0.979, Posterior=0.970
Infection rate=0.8, proportion infected given a positive test=0.987, Posterior=0.987
Infection rate=0.9, proportion infected given a positive test=0.994, Posterior=0.994
Infection rate=1.0, proportion infected given a positive test=1.000, Posterior=1.000
